# Exploring Spark with Pandas


Using pandas examples, convert the analysis to pyspark. This is useful if you discover your data grows too large for your tooling.

The purpose of this notebook is to familiarise yourself you the pyspark API. You are welcome to use the R version of this if you wish. As long as you are able to obtain the correct results. We will be using python in this notebook as it is quite widely used through data science and the community is very large.



#### Firstly, let's get our spark session

In [1]:
#congigure the platfrom
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.6 LTS
Release:	18.04
Codename:	bionic


In [2]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

In [3]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
# get spark 
VERSION='3.2.2'
!wget https://dlcdn.apache.org/spark/spark-$VERSION/spark-$VERSION-bin-hadoop3.2.tgz

--2022-08-15 15:44:09--  https://dlcdn.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 301112604 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.2-bin-hadoop3.2.tgz’

spark-3.2.2-bin-had 100%[===================>] 287.16M   255MB/s    in 1.1s    

2022-08-15 15:44:10 (255 MB/s) - ‘spark-3.2.2-bin-hadoop3.2.tgz’ saved [301112604/301112604]



In [5]:
# decompress spark
!tar xf spark-$VERSION-bin-hadoop3.2.tgz

# install python package to help with system paths
!pip install -q findspark

In [6]:
! ls -lht

total 288M
drwxr-xr-x  1 root root 4.0K Aug  3 20:21 sample_data
-rw-r--r--  1 root root 288M Jul 11 16:18 spark-3.2.2-bin-hadoop3.2.tgz
drwxr-xr-x 13 1000 1000 4.0K Jul 11 16:01 spark-3.2.2-bin-hadoop3.2


In [7]:
# Let Colab know where the java and spark folders are

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{VERSION}-bin-hadoop3.2"

In [8]:
# add pyspark to sys.path using findspark
import findspark
findspark.init()

# get a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
spark

In [10]:
from pyspark.sql import SparkSession
import pandas as pd 
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

### Overview


* Joining two dataframes/data sets
* Simple aggregations
* Persisting

#### JOIN: Pandas

We won't use this more in this notebook, but observe how the joins work.

We what happens if you change from the default inner join to outer joins.

In [11]:
customer_raw = [(1, 'bob', 3462543658686),
           (2, 'rob', 9087567565439),
           (3, 'tim', 5436586999467),
           (4, 'tom', 8349756853250)]

customer_cols = ['id', 'name', 'credit_card_number']



orders_raw = [(1, 'ketchup', 'bob', 1.20),
           (2, 'rutabaga', 'bob', 3.35),
           (3, 'fake vegan meat', 'rob', 13.99),
           (4, 'cheesey poofs', 'tim', 3.99),
           (5, 'ice cream', 'tim', 4.95),
           (6, 'protein powder', 'tom', 49.95)]

orders_cols = ['id', 'product_name', 'customer', 'price']

In [12]:
customer_df = pd.DataFrame(customer_raw, columns=customer_cols)
orders_df = pd.DataFrame(orders_raw, columns=orders_cols)

customer_df

joined_df = pd.merge(customer_df, orders_df, how='inner', left_on='name', right_on='customer')
joined_df

## For self study. What happens if (4, 'tom', 8349756853250) in valuesA becomes (4, 'tod', 8349756853250)
## How do the results change?
## More sensibly; what if customers have not made any orders but we still require them in the result set? 

,id_x,name,credit_card_number,id_y,product_name,customer,price
0,1,bob,3462543658686,1,ketchup,bob,1.20
1,1,bob,3462543658686,2,rutabaga,bob,3.35
2,2,rob,9087567565439,3,fake vegan meat,rob,13.99
3,3,tim,5436586999467,4,cheesey poofs,tim,3.99
4,3,tim,5436586999467,5,ice cream,tim,4.95
5,4,tom,8349756853250,6,protein powder,tom,49.95


#### JOIN: Spark

In [13]:
customersDF = spark.createDataFrame(customer_raw, customer_cols)

ordersDF = spark.createDataFrame(orders_raw, orders_cols)

# Show tables
customersDF.show()
ordersDF.show()

+---+----+------------------+
| id|name|credit_card_number|
+---+----+------------------+
|  1| bob|     3462543658686|
|  2| rob|     9087567565439|
|  3| tim|     5436586999467|
|  4| tom|     8349756853250|
+---+----+------------------+

+---+---------------+--------+-----+
| id|   product_name|customer|price|
+---+---------------+--------+-----+
|  1|        ketchup|     bob|  1.2|
|  2|       rutabaga|     bob| 3.35|
|  3|fake vegan meat|     rob|13.99|
|  4|  cheesey poofs|     tim| 3.99|
|  5|      ice cream|     tim| 4.95|
|  6| protein powder|     tom|49.95|
+---+---------------+--------+-----+



In [14]:
joinedDF = customersDF.join(ordersDF, customersDF.name == ordersDF.customer)
joinedDF.show()

+---+----+------------------+---+---------------+--------+-----+
| id|name|credit_card_number| id|   product_name|customer|price|
+---+----+------------------+---+---------------+--------+-----+
|  1| bob|     3462543658686|  1|        ketchup|     bob|  1.2|
|  1| bob|     3462543658686|  2|       rutabaga|     bob| 3.35|
|  2| rob|     9087567565439|  3|fake vegan meat|     rob|13.99|
|  3| tim|     5436586999467|  4|  cheesey poofs|     tim| 3.99|
|  3| tim|     5436586999467|  5|      ice cream|     tim| 4.95|
|  4| tom|     8349756853250|  6| protein powder|     tom|49.95|
+---+----+------------------+---+---------------+--------+-----+



## Simple Aggregations

Now let's explore simple aggregations. You will be using these often when doing exploratory work in big data. Remember, the intention here is that you grow familiar with the way the API works, and how to translate inquiries into that API.

> _How much did each person spend?_

In [15]:
joined_df.groupby('name').agg({"price": ["sum"]}) 

,price
,sum
name,
bob,4.55
rob,13.99
tim,8.94
tom,49.95


In [16]:
import pyspark.sql.functions as f

joinedDF.groupby('name').agg(f.sum('price').alias('total')).show()

+----+-----------------+
|name|            total|
+----+-----------------+
| bob|             4.55|
| rob|            13.99|
| tim|8.940000000000001|
| tom|            49.95|
+----+-----------------+



Let's use bigger data
  * NYC crash data

In [18]:
# save to the filesystem to prevent another load
! curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  391M    0  391M    0     0  3223k      0 --:--:--  0:02:04 --:--:-- 3782k


In [19]:
import pandas as pd
nyc_df = pd.read_csv('rows.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
# number or rows

print(len(nyc_df))

# this is quite large so we will work with a sample while we experiment in pandas as least.

1917681


We'll take a random sample at 20% of the original data

In [21]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [22]:
# we are also going to limit the columns to those we are going to work with

nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1', 
                       'BOROUGH', 'VEHICLE TYPE CODE 1', 
                       'NUMBER OF PERSONS INJURED']]

In [38]:
nyc_small.head(5)

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
797894,09/13/2017,Unspecified,NaN,Sedan,0.0
461101,03/23/2019,Following Too Closely,MANHATTAN,Station Wagon/Sport Utility Vehicle,0.0
1293045,07/24/2015,Unspecified,QUEENS,PASSENGER VEHICLE,0.0
482248,01/30/2019,Driver Inattention/Distraction,BROOKLYN,Sedan,0.0
1504308,07/15/2014,Driver Inexperience,BRONX,SPORT UTILITY / STATION WAGON,0.0


Now, let's create the pyspark dataframe. Now we two frames with the same content
  * nyc_small: pandas
  * sdf_small: pyspark

In [25]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out 
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf_small.schema)

/content/spark-3.2.2-bin-hadoop3.2/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


StructType(List(StructField(CRASH DATE,StringType,true),StructField(CONTRIBUTING FACTOR VEHICLE 1,StringType,true),StructField(BOROUGH,StringType,true),StructField(VEHICLE TYPE CODE 1,StringType,true),StructField(NUMBER OF PERSONS INJURED,StringType,true)))


# Questions

Answer the following questions by porting the pandas code to the Spark API



# Question 1


> On what day do most crashes occcur?

In [ ]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

In [35]:
### Spark?
from pyspark.sql.functions import sum,col,desc,count # import important aggregate functions

sdf_small.groupby('CRASH DATE').agg(count('CRASH DATE').alias("Total_per_day")).sort(desc("Total_per_day")).show(3)

+----------+-------------+
|CRASH DATE|Total_per_day|
+----------+-------------+
|01/21/2014|          256|
|11/15/2018|          202|
|01/07/2017|          194|
+----------+-------------+
only showing top 3 rows



# Question 2

> _Where do most crashes occur?_

In [26]:
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

BOROUGH
BROOKLYN         83897
QUEENS           71074
MANHATTAN        60108
BRONX            38494
STATEN ISLAND    11098
Name: BOROUGH, dtype: int64

In [36]:
## Spark?

sdf_small.groupby('BOROUGH').agg(count('BOROUGH').alias("Total")).sort(desc("Total")).show(5)

+---------+------+
|  BOROUGH| Total|
+---------+------+
|         |118865|
| BROOKLYN| 83897|
|   QUEENS| 71074|
|MANHATTAN| 60108|
|    BRONX| 38494|
+---------+------+
only showing top 5 rows



 # Question 3
 
 > What is the most common cause of accident in 'QUEENS'

In [37]:
nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

CONTRIBUTING FACTOR VEHICLE 1
Unspecified                       25758
Driver Inattention/Distraction    14987
Failure to Yield Right-of-Way      5957
Backing Unsafely                   3566
Following Too Closely              2541
Name: CONTRIBUTING FACTOR VEHICLE 1, dtype: int64

In [53]:
sdf_small_update = sdf_small.filter(f.col("BOROUGH")=='QUEENS')
sdf_small_update.show(5)

+----------+-----------------------------+-------+-------------------+-------------------------+
|CRASH DATE|CONTRIBUTING FACTOR VEHICLE 1|BOROUGH|VEHICLE TYPE CODE 1|NUMBER OF PERSONS INJURED|
+----------+-----------------------------+-------+-------------------+-------------------------+
|07/24/2015|                  Unspecified| QUEENS|  PASSENGER VEHICLE|                      0.0|
|12/27/2016|                  Unspecified| QUEENS|              Sedan|                      0.0|
|09/16/2018|         Driver Inattentio...| QUEENS|              Sedan|                      0.0|
|01/14/2020|         Driver Inattentio...| QUEENS|                   |                      1.0|
|06/23/2013|                  Unspecified| QUEENS|  PASSENGER VEHICLE|                      0.0|
+----------+-----------------------------+-------+-------------------+-------------------------+
only showing top 5 rows



In [55]:
## Spark?
sdf_small_update.groupby('CONTRIBUTING FACTOR VEHICLE 1').agg(count('CONTRIBUTING FACTOR VEHICLE 1').alias("Total")).sort(desc("Total")).show(5)

+-----------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1|Total|
+-----------------------------+-----+
|                  Unspecified|25758|
|         Driver Inattentio...|14987|
|         Failure to Yield ...| 5957|
|             Backing Unsafely| 3566|
|         Following Too Clo...| 2541|
+-----------------------------+-----+
only showing top 5 rows



In [41]:
from pyspark.sql import SparkSession

# Question 4

> _What is the average number or injuries for specific cars driving in specific suburbs_


In [56]:
nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

VEHICLE TYPE CODE 1  BOROUGH  
12 passage           BROOKLYN     5.0
amb                  MANHATTAN    4.0
PICKUP               BRONX        4.0
Name: NUMBER OF PERSONS INJURED, dtype: float64

In [61]:
## Spark?
from pyspark.sql.functions import sum,col,desc,count,avg # import important aggregate functions
sdf_small.groupby('VEHICLE TYPE CODE 1','BOROUGH').agg(avg('NUMBER OF PERSONS INJURED').alias("Total")).sort(desc("Total")).show(3)

+-------------------+---------+-----+
|VEHICLE TYPE CODE 1|  BOROUGH|Total|
+-------------------+---------+-----+
|         12 passage| BROOKLYN|  5.0|
|              BACKH|   QUEENS|  4.0|
|                amb|MANHATTAN|  4.0|
+-------------------+---------+-----+
only showing top 3 rows

